DH Parameter solver

In [2]:
import numpy as np
import sympy as sp

In [3]:
c = lambda angle: round(np.cos(angle),2)
s = lambda angle: round(np.sin(angle),2)

In [4]:
def DH_matrix(theta,alpha,a,d):
    mat = []
    for i in range(len(theta)):
        mat.append([round(np.deg2rad(theta[i]),2), np.round(np.deg2rad(alpha[i]),2), a[i],d[i]])
    return np.array(mat)

In [5]:
T = lambda dh:np.array([
    [c(dh[0]), -s(dh[0])*c(dh[1]), s(dh[0])*s(dh[1]), dh[2]*c(dh[0])],
    [s(dh[0]), c(dh[0])*c(dh[1]), -c(dh[0])*s(dh[1]), dh[2]*s(dh[0])],
    [0, s(dh[1]), c(dh[1]), dh[3]],
    [0,0,0,1]
    ])

In [6]:
def EE(dh, T):
    ans = []
    p = []
    I = np.eye(4)
    for i in range(len(dh)):
        I = I@T(dh[i,:])
        p.append(I[:3,3])
        ans = I
    return ans,p

In [7]:
def jacobian_calculator(p, config):
    J = []
    Z_i = np.array([0, 0, 1])  # Initial Z axis for the base frame
    O_i = np.array([0, 0, 0])  # Initial origin for the base frame
    
    for i in range(len(config)):
        j_v = np.zeros(3)
        j_w = np.zeros(3)
        O_j = p[i]  # Position vector of the i-th joint
        
        if config[i].lower() == 'r':    # Revolute joint
            j_v = np.cross(Z_i, (O_j - O_i))
            j_w = Z_i
        elif config[i].lower() == 'p':  # Prismatic joint
            j_v = Z_i
            j_w = np.zeros(3)

        # Combined Jacobian matrix
        J_i = np.hstack((j_v, j_w))
        J.append(J_i)

        # Update Z_i and O_i for the next joint
        if i < len(config) - 1:
            Z_i = np.array([0, 0, 1])  
            O_i = O_j                   # Update origin 

    J = np.array(J).T
    return J

In [8]:
def EE_velocity(J, joint_velocity):
    return J[:3, :]@np.array(joint_velocity).T

SCARA 

In [9]:
theta = [60,30,0,10]
alpha = [0,180,0,0]
a = [1,2,0,0]
d = [0,0,5,3]
joint_velocity = [0.5, 0.6, 0.7, 0.8]

In [10]:
a = DH_matrix(theta,alpha,a,d)

In [11]:
T_0n, p = EE(a, T)
T_0n

array([[ 0.171173,  0.996831,  0.      ,  0.5     ],
       [ 0.996831, -0.171173,  0.      ,  2.8838  ],
       [ 0.      ,  0.      , -1.      , -8.      ],
       [ 0.      ,  0.      ,  0.      ,  1.      ]])

In [12]:
J_scara = jacobian_calculator(p, "RRPR")
J_scara

array([[-0.87  , -2.0138,  0.    , -0.    ],
       [ 0.5   ,  0.    ,  0.    ,  0.    ],
       [ 0.    ,  0.    ,  1.    ,  0.    ],
       [ 0.    ,  0.    ,  0.    ,  0.    ],
       [ 0.    ,  0.    ,  0.    ,  0.    ],
       [ 1.    ,  1.    ,  0.    ,  1.    ]])

In [13]:
EE_velocity(J_scara, joint_velocity)

array([-1.64328,  0.25   ,  0.7    ])

Question 9 - PPP Configuration

In [14]:
theta_9 = [0,90,0]
alpha_9 = [-90, 90, 0]
a_9 = [0,0,0]
d_9 = [3,4,5]
jv = [0.1,.2,3]

In [25]:
dh_9 = DH_matrix(theta_9, alpha_9, a_9, d_9)
dh_9

array([[ 0.  , -1.57,  0.  ,  3.  ],
       [ 1.57,  1.57,  0.  ,  4.  ],
       [ 0.  ,  0.  ,  0.  ,  5.  ]])

In [17]:
T_9,p_9 = EE(dh_9, T)

In [19]:
J_9= jacobian_calculator(p_9,"PPP")
J_9

array([[0., 0., 0.],
       [0., 0., 0.],
       [1., 1., 1.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [20]:
EE_velocity(J_9, jv)

array([0. , 0. , 3.3])

Question 10 - RRRRR Configuration

In [21]:
a_10 = [0,0,0,0,0,0,0]
d_10 = [0, 2,3,4,5,6,7]
theta_10 = [10,20,30,40,50,60,70]
alpha_10 = [0, 90,0,0,-90,0,0]

In [22]:
dh_10 = DH_matrix(theta_10,alpha_10,a_10,d_10)
dh_10

array([[ 0.17,  0.  ,  0.  ,  0.  ],
       [ 0.35,  1.57,  0.  ,  2.  ],
       [ 0.52,  0.  ,  0.  ,  3.  ],
       [ 0.7 ,  0.  ,  0.  ,  4.  ],
       [ 0.87, -1.57,  0.  ,  5.  ],
       [ 1.05,  0.  ,  0.  ,  6.  ],
       [ 1.22,  0.  ,  0.  ,  7.  ]])

In [23]:
T_10,p_10 = EE(dh_10,T)
T_10

array([[ -0.10106176,   0.65148534,  -0.7496165 ,  -3.78821449],
       [  0.82711617,  -0.3777612 ,  -0.42634009, -16.01602116],
       [ -0.5563721 ,  -0.65771805,  -0.4936    ,  -4.4168    ],
       [  0.        ,   0.        ,   0.        ,   1.        ]])

In [24]:
p_10

[array([0., 0., 0.]),
 array([0., 0., 2.]),
 array([ 1.4892, -2.6184,  2.    ]),
 array([ 3.4748, -6.1096,  2.    ]),
 array([  5.9568, -10.4736,   2.    ]),
 array([  1.459101  , -13.03164054,  -0.9616    ]),
 array([ -3.78821449, -16.01602116,  -4.4168    ])]